In [1]:
import os
import numpy as np
import torch
from transformers import CLIPProcessor, CLIPModel
from PIL import Image

/Users/VoThinhPhat/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14-336").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14-336")

/Users/VoThinhPhat/Library/Python/3.9/lib/python/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
image_folder = "/Users/VoThinhPhat/Desktop/chatKPT-2024-AIC-HCMC/data/batch1/keyframes/keyframes_L01/L01_V001"

In [4]:
image_files = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
image_files.sort()

In [5]:
image_embeddings = []
for image_path in image_files:
    try:
        # Open and preprocess the image
        image = Image.open(image_path).convert("RGB")
        inputs = processor(images=image, return_tensors="pt").to(device)

        # Generate image embedding
        with torch.no_grad():
            embedding = model.get_image_features(**inputs).cpu().numpy().flatten()  # Flatten to make it a 1D array

        # Append the embedding to the list
        image_embeddings.append(embedding)
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        
image_embeddings_array = np.array(image_embeddings)

In [6]:
np.save("L01_V001.npy", image_embeddings_array)